<a href="https://colab.research.google.com/github/btwwang1/btwwang1/blob/main/bus_tracker_(combined).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#bus tracker combined v.2
import requests
import json
import datetime
import pytz
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

check = True
CTB_check = True
KMB_check = True
route = input("Enter the bus route: ")
route = route.upper()
directions = ('inbound', 'outbound')
stopid_list_main = []
stop_name_list_main = []

def handle_click_CTB(index, bound):
    # Get the associated row data
    stop_name = df_main[bound].iloc[index]['Stop Names']
    stop_id = df_main[bound].iloc[index]['Stop IDs']
    list_count = df_main[bound].iloc[index]['Count']
    display_eta_CTB(stop_name, stop_id, route, bound, list_count)

def display_eta_CTB(stop_name, stop_id, route, bound, list_count):
    eta_url = f"https://rt.data.gov.hk/v2/transport/citybus/eta/CTB/{stop_id}/{route}"
    eta_response = requests.get(eta_url)
    if eta_response.status_code == 200:
        retrieved = json.loads(eta_response.text)
        data = retrieved["data"]
        eta_list_o = []
        eta_list_i = []
        for entries in data:
            if entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "O" and entries["route"] == route:
                eta_form = datetime.datetime.fromisoformat(entries["eta"])
                eta_list_o.append(eta_form)
            elif entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "I" and entries["route"] == route:
                eta_form = datetime.datetime.fromisoformat(entries["eta"])
                eta_list_i.append(eta_form)
        remaining_time_o = []
        remaining_time_i = []
        for eta in eta_list_o:
            remaining_time_o.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
        for eta in eta_list_i:
            remaining_time_i.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
        remaining_time_main = (remaining_time_i, remaining_time_o)
        remaining_time_i = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_i]
        remaining_time_o = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_o]
        if remaining_time_i != []:
            for remaining_time in remaining_time_i:
                print(f"Remaining time for route {route} to {list((stop_name_list_main[list_count - 1]).values())[-1]} operated by CTB at stop {stop_name} is {remaining_time}")
        elif remaining_time_o != []:
            for remaining_time in remaining_time_o:
                print(f"Remaining time for route {route} to {list((stop_name_list_main[list_count - 1]).values())[-1]} operated by CTB at stop {stop_name} is {remaining_time}")
        else:
            print("There is no bus operating at this stop in the coming hour.")
    else:
        print(f'Error: Unable to retrieve ETAs for route {route}')

def handle_click(index, service_type):
    # Get the associated row data
    stop_name = df_main[service_type].iloc[index]['Stop Names']
    stop_id = df_main[service_type].iloc[index]['Stop IDs']
    list_count = df_main[service_type].iloc[index]['Count']
    display_eta(stop_name, stop_id, route, service_type, list_count)

def display_eta(stop_name, stop_id, route, service_type, list_count):
    count = 0
    eta_url = f"https://data.etabus.gov.hk/v1/transport/kmb/stop-eta/{stop_id}"
    eta_response = requests.get(eta_url)
    if eta_response.status_code == 200:
        retrieved = json.loads(eta_response.text)
        eta_data = retrieved["data"]
        eta_list = []
        st = service_type.upper()
        for entries in eta_data:
            if entries["route"] == route and str(entries["service_type"]) == service_type[-1] and entries["dir"] == st[0]:
                    eta_form = datetime.datetime.fromisoformat(entries["eta"])
                    eta_list.append(eta_form)
        remaining_time_main = []
        for eta in eta_list:
            remaining_time_main.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
        remaining_time_main = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_main]
        if remaining_time_main != []:
            for remaining_time in remaining_time_main:
                    print(f"Remaining time for route {route} to {list((stop_name_list_main[list_count - 1]).values())[-1]} operated by KMB/LWB at stop {stop_name} is {remaining_time}")
        else:
            print(f'There is no service at the stop by KMB in the coming hour')
    else:
        print(f'Error: Unable to retrieve ETAs for route {route}')

stop_table = {}
stop_table_main = {}
df = {}
df_main = {}
count = 0
list_count = 0
for bound in directions:
    if check == False and CTB_check == True:
      for bound in directions:
        url = f"https://rt.data.gov.hk/v2/transport/citybus/route-stop/CTB/{route}/{bound}"
        response = requests.get(url)
        if response.status_code == 200:
            stop_name_list = {}
            # Parse the JSON response and extract the stop information
            data = json.loads(response.text)
            route_list = data["data"]
            # Display the stop information to the user
            for stop in route_list:
                stop_id = stop['stop']
                url_stop = f"https://rt.data.gov.hk/v2/transport/citybus/stop/{stop_id}"
                response_stop = requests.get(url_stop)
                if response_stop.status_code == 200:
                    # Parse the JSON response and extract the stop information
                    data_stop = json.loads(response_stop.text)
                    stop_name_list[stop_id] = data_stop["data"]["name_en"]
                    # Display the stop information to the user
            stop_name_list_main.append(stop_name_list)
            if stop_name_list == {}:
              CTB_check = False
              break
            print(f"\nStop names for route {route} (CTB) \n{list(stop_name_list.values())[0]} \nTO \n{list(stop_name_list.values())[-1]}")
            list_count += 1
            # Create a Pandas DataFrame from the data
            stop_table = {"Stop Names": list(stop_name_list.values()), "Stop IDs": list(stop_name_list.keys()), "Bound": bound, "Count" : list_count}
            stop_table_main[bound] = stop_table
            df = pd.DataFrame(stop_table)
            df_main[bound] = df
            # Create buttons for each row
            buttons = []
            for i in range(len(df_main[bound])):
                button = widgets.Button(description=df_main[bound].iloc[i]['Stop Names'])
                button.layout.width = "300px"  # Adjust the button width here
                buttons.append(button)

            # Create a VBox layout for the buttons
            vbox = widgets.VBox(buttons)

            # Display the buttons
            display(vbox)

            # Add button click event handlers
            for i in range(len(buttons)):
                buttons[i].on_click(lambda event, index=i, bound=bound: handle_click_CTB(index, bound))
        else:
            print(f'Error: Unable to retrieve route information for route {route}')

    for count in range(10):
      if check == True:
        for bound in directions:
          url = f"https://data.etabus.gov.hk/v1/transport/kmb/route-stop/{route}/{bound}/{count + 1}"
          response = requests.get(url)
          if response.status_code == 200:
              stop_name_list = {}
              stopid_list = []
              service_type_list = {}
              # Parse the JSON response and extract the stop information
              data = json.loads(response.text)
              route_list = data["data"]
              # Display the stop information to the user
              for stop in route_list:
                  service_type = bound + stop['service_type']
                  stop_id = stop['stop']
                  url_stop = f"https://data.etabus.gov.hk/v1/transport/kmb/stop/{stop_id}"
                  response_stop = requests.get(url_stop)
                  if response_stop.status_code == 200:
                      # Parse the JSON response and extract the stop information
                      data_stop = json.loads(response_stop.text)
                      stop_name_list[stop_id] = data_stop["data"]["name_en"]
                      service_type_list[stop_id] = service_type
                      # Display the stop information to the user
              if stop_name_list == {} and count == 0:
                  KMB_check = False
                  check = False
                  break
              if stop_name_list != {}:
                  stop_name_list_main.append(stop_name_list)
              else:
                check = False
                break
              print(f"\nStop names for route {route} (KMB) \n{list(stop_name_list.values())[0]}\nTO \n{list(stop_name_list.values())[-1]}")
              list_count += 1
              # Create a Pandas DataFrame from the data
              sub_stop_table = {"Stop Names": list(stop_name_list.values()), "Stop IDs": list(stop_name_list.keys()), "Service Type": service_type, "Count" : list_count}
              stop_table[service_type] = sub_stop_table
              df = pd.DataFrame(sub_stop_table)
              df_main[service_type] = df
              # Create buttons for each row
              buttons = []
              for i in range(len(df_main[service_type])):
                  button = widgets.Button(description=df_main[service_type].iloc[i]['Stop Names'])
                  button.layout.width = "300px"  # Adjust the button width here
                  buttons.append(button)

              # Create a VBox layout for the buttons
              vbox = widgets.VBox(buttons)

              # Display the buttons
              display(vbox)

              # Add button click event handlers
              for i in range(len(buttons)):
                  buttons[i].on_click(lambda event, index=i, service_type=service_type: handle_click(index, service_type))
          else:
              print(f"Error: Unable to retrieve routing for route {route}")
if KMB_check == False and CTB_check == False:
      print("There is no such route operated by CTB/KMB/LWB.")

Enter the bus route: 101

Stop names for route 101 (KMB) 
KENNEDY TOWN BUS TERMINUS
TO 
KWUN TONG (YUE MAN SQUARE) BUS TERMINUS



Stop names for route 101 (KMB) 
KWUN TONG (YUE MAN SQUARE) BUS TERMINUS
TO 
KENNEDY TOWN BUS TERMINUS



Stop names for route 101 (KMB) 
RUMSEY STREET
TO 
KWUN TONG (YUE MAN SQUARE) BUS TERMINUS



Stop names for route 101 (KMB) 
KWUN TONG (YUE MAN SQUARE) BUS TERMINUS
TO 
THE LANDMARK



Stop names for route 101 (CTB) 
Kwun Tong (Yue Man Square) 
TO 
Kennedy Town



Stop names for route 101 (CTB) 
Kennedy Town 
TO 
Kwun Tong (Yue Man Square)


There is no bus operating at this stop in the coming hour.
Remaining time for route 101 to Kennedy Town at stop Pacific Place, Queensway is 0:01:24.460926
Remaining time for route 101 to Kennedy Town at stop Pacific Place, Queensway is 0:18:01.460879
Remaining time for route 101 to Kennedy Town at stop Pacific Place, Queensway is 0:18:54.460865
Remaining time for route 101 to Kennedy Town at stop Central Market, Queen Victoria Street is 0:01:49.190598
Remaining time for route 101 to Kennedy Town at stop Central Market, Queen Victoria Street is 0:06:49.190566
Remaining time for route 101 to Kennedy Town at stop Central Market, Queen Victoria Street is 0:23:58.190557
Remaining time for route 101 to THE LANDMARK at stop KAI YIP ESTATE is 0:03:12.340817
Remaining time for route 101 to THE LANDMARK at stop KAI YIP ESTATE is 0:13:21.340783
Remaining time for route 101 to THE LANDMARK at stop KAI YIP ESTATE is 0:24:57.340774
Remaining time for route 101 to THE LANDMARK at stop THE CHARTERHOUS

In [ ]:
#bus tracker combined
import requests
import json
import datetime
import pytz
import pandas as pd

route = input("Enter the bus route: ")
directions = ('inbound', 'outbound')
stopid_list_ctb = []
stopid_list_kmb = []
stop_name_list_ctb = []
stop_name_list_kmb = []

for bound in directions:
    url = f"https://rt.data.gov.hk/v2/transport/citybus/route-stop/CTB/{route}/{bound}"
    response = requests.get(url)
    if response.status_code == 200:
        stopid_list = []
        # Parse the JSON response and extract the stop information
        data = json.loads(response.text)
        route_list = data["data"]
        # Display the stop information to the user
        for stop in route_list:
            stopid_list.append(stop['stop'])
        stopid_list_ctb.append(stopid_list)
        stop_name_list = {}
        for stop_id in stopid_list:
            url_stop = f"https://rt.data.gov.hk/v2/transport/citybus/stop/{stop_id}"
            response_stop = requests.get(url_stop)
            if response_stop.status_code == 200:
                # Parse the JSON response and extract the stop information
                data_stop = json.loads(response_stop.text)
                stop_name_list[stop_id] = data_stop["data"]["name_en"]
                # Display the stop information to the user
        print(f"\n CTB Stop names for route {route} departing from {stop_name_list[stopid_list[0]]} to {stop_name_list[stopid_list[-1]]}:")
            # Create a Pandas DataFrame from the data
        stop_table = {"Stops ID": list(stop_name_list.keys()), "Stop Names": list(stop_name_list.values())}
        df = pd.DataFrame(stop_table)
        # Set the 'Name' column as the index
        df.set_index('Stops ID', inplace=True)

        # Print the DataFrame as a table
        print(df)
        stop_name_list_ctb.append(stop_name_list)
    else:
        print(f"Error: Unable to retrieve routing for route {route}")
for bound in directions:
  for count in range(10):
    url = f"https://data.etabus.gov.hk/v1/transport/kmb/route-stop/{route}/{bound}/{count + 1}"
    response = requests.get(url)
    if response.status_code == 200:
          stopid_list = []
          # Parse the JSON response and extract the stop information
          data = json.loads(response.text)
          route_list = data["data"]
          # Display the stop information to the user
          for stop in route_list:
              stopid_list.append(stop['stop'])
          if stopid_list == []: break
          stopid_list_kmb.append(stopid_list)
          stop_name_list = {}
          for stop_id in stopid_list:
              url_stop = f"https://data.etabus.gov.hk/v1/transport/kmb/stop/{stop_id}"
              response_stop = requests.get(url_stop)
              if response_stop.status_code == 200:
                  # Parse the JSON response and extract the stop information
                  data_stop = json.loads(response_stop.text)
                  stop_name_list[stop_id] = data_stop["data"]["name_en"]
                  # Display the stop information to the user

          print(f"\n KMB Stop names for route {route} departing from {stop_name_list[stopid_list[0]]} to {stop_name_list[stopid_list[-1]]}:")
              # Create a Pandas DataFrame from the data
          stop_table = {"Stops ID": list(stop_name_list.keys()), "Stop Names": list(stop_name_list.values())}
          df = pd.DataFrame(stop_table)
          # Set the 'Name' column as the index
          df.set_index('Stops ID', inplace=True)
          # Print the DataFrame as a table
          print(df)
          stop_name_list_kmb.append(stop_name_list)

    else:
          print(f"Error: Unable to retrieve routing for route {route}")
stopid_list_all = [stopid_list_ctb, stopid_list_kmb]
stop_name_list_all = [stop_name_list_ctb, stop_name_list_kmb]
check = True
while check == True:
    trace_stop = input("Please input the stop id u would like to trace: ")
    for stop_id_items in stopid_list_all:
      if len(trace_stop) != 6:
        for stopid_lists in stop_id_items:
          eta_url = f"https://data.etabus.gov.hk/v1/transport/kmb/stop-eta/{trace_stop}"
          eta_response = requests.get(eta_url)
          if eta_response.status_code == 200:
              retrieved = json.loads(eta_response.text)
              data = retrieved["data"]
              eta_list_o = []
              eta_list_i = []
              for entries in data:
                try:
                  if entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "O" and entries["route"] == route:
                      eta_form = datetime.datetime.fromisoformat(entries["eta"])
                      eta_list_o.append(eta_form)
                  elif entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "I" and entries["route"] == route:
                      eta_form = datetime.datetime.fromisoformat(entries["eta"])
                      eta_list_i.append(eta_form)
                except:
                      print(f'There is no service at the stop operated by KMB in the coming hour')
              remaining_time_o = []
              remaining_time_i = []
              for eta in eta_list_o:
                  remaining_time_o.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
              for eta in eta_list_i:
                  remaining_time_i.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
              remaining_time_main = (remaining_time_i, remaining_time_o)
              remaining_time_i = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_i]
              remaining_time_o = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_o]
              count = 0
              if remaining_time_i != []:
                for remaining_time in remaining_time_i:
                  if count <= 5 and trace_stop in stopid_list_kmb[count]:
                    print(f"Remaining time for route {route} to {list((stop_name_list_kmb[count]).values())[-1]} at stop {stop_name_list_kmb[count][trace_stop]} is {remaining_time}")
                    count += 1
              elif remaining_time_o != []:
                for remaining_time in remaining_time_o:
                  if count <= 5 and trace_stop in stopid_list_kmb[count]:
                    print(f"Remaining time for route {route} to {list((stop_name_list_kmb[count]).values())[-1]} at stop {stop_name_list_kmb[count][trace_stop]} is {remaining_time}")
                    count += 1
          else:
              print(f'Error: Unable to retrieve ETAs for route {route}')
      else:
        for stopid_lists in stop_id_items:
          eta_url = f"https://rt.data.gov.hk/v2/transport/citybus/eta/CTB/{trace_stop}/{route}"
          eta_response = requests.get(eta_url)
          if eta_response.status_code == 200:
              retrieved = json.loads(eta_response.text)
              data = retrieved["data"]
              eta_list_o = []
              eta_list_i = []
              for entries in data:
                  if entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "O" and entries["route"] == route:
                      eta_form = datetime.datetime.fromisoformat(entries["eta"])
                      eta_list_o.append(eta_form)
                  elif entries["rmk_en"] != "KMB Cycle" and entries["dir"] == "I" and entries["route"] == route:
                      eta_form = datetime.datetime.fromisoformat(entries["eta"])
                      eta_list_i.append(eta_form)
              remaining_time_o = []
              remaining_time_i = []
              for eta in eta_list_o:
                  remaining_time_o.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
              for eta in eta_list_i:
                  remaining_time_i.append(abs(eta - datetime.datetime.now(pytz.timezone('Asia/Hong_Kong'))))
              remaining_time_main = (remaining_time_i, remaining_time_o)
              remaining_time_i = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_i]
              remaining_time_o = [str(datetime.timedelta(seconds=t.seconds, microseconds=t.microseconds)) for t in remaining_time_o]
              if remaining_time_i != []:
                for remaining_time in remaining_time_i:
                    print(f"Remaining time for route {route} to {list((stop_name_list_ctb[0]).values())[-1]} at stop {stop_name_list_ctb[0][trace_stop]} is {remaining_time}")
              elif remaining_time_o != []:
                for remaining_time in remaining_time_o:
                    print(f"Remaining time for route {route} to {list((stop_name_list_ctb[1]).values())[-1]} at stop {stop_name_list_ctb[1][trace_stop]} is {remaining_time}")
              else:
                print("There is no bus operating at this stop by CTB in the coming hour.")
          else:
              print(f'Error: Unable to retrieve ETAs for route {route}')

      check = input("Do you want to trace another Stop? T/F ")
      check = check.upper()
      if check == "T":
           check = True
      else:
          break